<a href="https://colab.research.google.com/github/SiriuXProtocoL/Tensorflow_examples/blob/main/04_regression_fuel_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Basic regression: Predict fuel efficiency
- We will use the Auto MPG Dataset from the UCI Machine Learning Repository and builds a model to predict the fuel efficiency of late-1970s and early 1980s automobile

In [ ]:
# Use seaborn for pairplot
!pip install -q seaborn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)


###Load the Data
- First download and import the dataset using pandas

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [ ]:
dataset = raw_dataset.copy()
dataset.tail()

###Clean the data
- Check for null values

In [ ]:
dataset.isna().sum()

- Lets just drop those rows in Horsepower
- othe methods like mean, median, mode replacements can also be adopted

In [ ]:
dataset = dataset.dropna()

- The Origin column is a categorical column not a numeric one so we will have to convert that using the one hot encoding vector

In [ ]:
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [ ]:
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

###Split the data into train and test


In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

###Inspect the data
- Have a quick look at the joint distribution of a few pairs of columns from the training set.

- Looking at the top row it should be clear that the fuel efficiency (MPG) is a function of all the other parameters. Looking at the other rows it should be clear that they are each functions of eachother.

In [ ]:
sns.pairplot(train_dataset[['MPG', 'Cylinders', 'Displacement', 'Weight']], diag_kind='kde')

In [ ]:
train_dataset.describe().transpose()

###Split features from labels
- Separate the target value, the "label", from the features. This label is the value that you will train the model to predict.

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

###Normalization
- It is good practice to normalize features that use different scales and ranges.

- One reason this is important is because the features are multiplied by the model weights. So the scale of the outputs and the scale of the gradients are affected by the scale of the inputs.

- Although a model might converge without feature normalization, normalization makes training much more stable. 

- We can observe that all the values are at different scales

In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

- The preprocessing.Normalization layer is a clean and simple way to build that preprocessing into your model.

- The first step is to create the layer:

In [ ]:
normalizer = preprocessing.Normalization()

- Then .adapt() it to the data:

In [ ]:
normalizer.adapt(np.array(train_features))

- This calculates the mean and variance, and stores them in the layer. 

In [ ]:
print(normalizer.mean.numpy())

- When the layer is called it returns the input data, with each feature independently normalized:

In [ ]:
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

###Linear regression
**One Variable**
- A single-variable linear regression, to predict MPG from Horsepower
  - Normalize the input horsepower.
  - Apply a linear transformation (y = mx + c) to produce 1 output using layers.Dense
- The number of inputs can either be set by the input_shape argument, or automatically when the model is run for the first time.

- First create the horsepower Normalization layer:

In [ ]:
horsepower = np.array(train_features['Horsepower'])

horsepower_normalizer = preprocessing.Normalization(input_shape=[1,])
horsepower_normalizer.adapt(horsepower)

- Build the sequential model:

In [ ]:
horsepower_model = tf.keras.Sequential([
    horsepower_normalizer,
    layers.Dense(units=1)
])

horsepower_model.summary()

- This model will predict MPG from Horsepower.

- Run the untrained model on the first 10 horse-power values. The output won't be good, but you'll see that it has the expected shape, (10,1):

In [ ]:
horsepower_model.predict(horsepower[:10])

- Once the model is built, configure the training procedure using the Model.compile() method. 
- The most important arguments to compile are the loss and the optimizer since these define what will be optimized (mean_absolute_error) and how (using the optimizers.Adam).

In [ ]:
horsepower_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

- Once the training is configured, use Model.fit() to execute the training

In [ ]:
%%time
history = horsepower_model.fit(
    train_features['Horsepower'], train_labels,
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)


- Visualize the model's training progress using the stats stored in the history object.

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

- Let have a plot


In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)


In [ ]:
plot_loss(history)


- Collect the results on the test set, for later

In [ ]:
test_results = {}

test_results['horsepower_model'] = horsepower_model.evaluate(
    test_features['Horsepower'],
    test_labels, verbose=0)

- SInce this is a single variable regression it's easy to look at the model's predictions as a function of the input

In [ ]:
x = tf.linspace(0.0, 250, 251)
y = horsepower_model.predict(x)

In [ ]:
def plot_horsepower(x, y):
  plt.scatter(train_features['Horsepower'], train_labels, label='Data')
  plt.plot(x, y, color='k', label='Predictions')
  plt.xlabel('Horsepower')
  plt.ylabel('MPG')
  plt.legend()


In [ ]:
plot_horsepower(x,y)

**Multiple inputs**
- We can use an almost identical setup (y=mx + c) to make predictions based on multiple inputs. This model still does the same except that m is a matrix and c is a vector.

- This time use the Normalization layer that was adapted to the whole dataset.

In [ ]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

- When you call this model on a batch of inputs, it produces units=1 outputs for each example.

In [ ]:
linear_model.predict(train_features[:10])

- When you call the model it's weight matrices will be built. Now you can see that the kernel (the m in y = mx + c) has a shape of (9,1).

In [ ]:
linear_model.layers[1].kernel


- Use the same compile and fit calls as for the single input horsepower model:

In [ ]:
linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')


In [ ]:
%%time
history = linear_model.fit(
    train_features, train_labels, 
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)


- Using all the inputs achieves a much lower training and validation error than the horsepower model

In [ ]:
plot_loss(history)

- Collect the results on the test set, for later

In [ ]:
test_results['linear_model'] = linear_model.evaluate(
    test_features, test_labels, verbose=0)


###A DNN regression

> The code is basically the same except the model is expanded to include some "hidden" non-linear layers.

- These models will contain a few more layers than the linear model:

    - The normalization layer.
    - Two hidden, nonlinear, Dense layers using the relu nonlinearity.
    - A linear single-output layer.

- Both will use the same training procedure so the compile method is included in the build_and_compile_model function below.





In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model


**One variable**
- Start with a DNN model for a single input: "Horsepower"

In [ ]:
dnn_horsepower_model = build_and_compile_model(horsepower_normalizer)

- This model has quite a few more trainable parameters than the linear models.

In [ ]:
dnn_horsepower_model.summary()


- Train the model:

In [ ]:
%%time
history = dnn_horsepower_model.fit(
    train_features['Horsepower'], train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

- This model does slightly better than the linear-horsepower model.

In [ ]:
plot_loss(history)


- If you plot the predictions as a function of Horsepower, you'll see how this model takes advantage of the nonlinearity provided by the hidden layers:

In [ ]:
x = tf.linspace(0.0, 250, 251)
y = dnn_horsepower_model.predict(x)


In [ ]:
plot_horsepower(x, y)


- Collect the results on the test set, for later

In [ ]:
test_results['dnn_horsepower_model'] = dnn_horsepower_model.evaluate(
    test_features['Horsepower'], test_labels,
    verbose=0)


###Full Model
**Multiple variable**

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()


In [ ]:
%%time
history = dnn_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)


In [ ]:
plot_loss(history)

- Collect the results on the test set

In [ ]:
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

###Performance
- Now that all the models are trained check the test-set performance and see how they did

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T

- These results match the validation error seen during training
###MAke Predictions
- Finally, predict have a look at the errors made by the model when making predictions on the test set

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)


- It looks like the model predicts reasonably well.

- Now take a look at the error distribution:

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [MPG]')
_ = plt.ylabel('Count')


- If you're happy with the model save it for later use

In [ ]:
dnn_model.save('dnn_model')

- If you reload the model, it gives identical output

In [ ]:
reloaded = tf.keras.models.load_model('dnn_model')

test_results['reloaded'] = reloaded.evaluate(
    test_features, test_labels, verbose=0)


In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T